In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
from processing import *

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)
pd.set_option('mode.chained_assignment',  None)

get_in_off_merged_data = pd.read_csv('./data/get_in_off_bus.csv')

# getrouteinfoall
getrouteinfoall = pd.read_csv('./data/getRouteInfoAll.csv', encoding = 'cp949', index_col = 0)
getrouteinfoall_df = getrouteinfoall[['ROUTE_CD','ROUTE_NO']]

### gestationbyrouteall
getstationbyrouteall = pd.read_csv('./data/getStationByRouteAll.csv', encoding = 'cp949', index_col = 0)
getstationbyrouteall_df1 = getstationbyrouteall[['ROUTE_CD','BUSSTOP_SEQ','BUSSTOP_TP','GPS_LATI','GPS_LONG']]
getstationbyrouteall_df2 = getstationbyrouteall[['ROUTE_CD','BUS_STOP_ID','BUSSTOP_SEQ','BUSSTOP_TP','GPS_LATI','GPS_LONG']]

In [74]:
get_in_off_merged_data_copy.shape

(46114, 23)

In [2]:
get_in_off_merged_data_copy = data_preprocessing.get_in_off_merged_data_preprocessing(get_in_off_merged_data)
get_in_off_merged_data_copy.head()

,노선,정류장순번,정류장,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,DAY
0,102,1,수통골기점지,0,0,0,0,0,0,0,0,0,7,0,0,0,0,0,0,0,6,0,1
1,102,2,수통골삼거리,1,2,3,6,5,4,4,5,7,9,13,8,8,8,3,5,4,1,0,1
2,102,3,한밭대학교,1,3,6,11,9,9,27,18,16,26,40,45,32,33,9,20,12,16,0,1
3,102,4,삼성화재연수원,0,0,3,9,0,10,30,21,12,33,35,49,34,34,10,26,24,16,0,1
4,102,5,하우스토리/신협연수원,2,5,17,14,12,14,31,22,18,36,40,52,36,37,11,25,26,10,0,1


---

# 히트맵 그리기

In [3]:
# 각 노선별로 평균 데이터프레임 생성하는 코드 ex) AVERAGE_DATA_102 : 102번 버스의 평균값
bus_numbers = [102, 103, 105, 106, 108, 115, 119, 201, 211, 301, 311, 314, 511, 603, 604, 605, 613, 615, 703, 704, 705, 706, 802]
average_data_dict = {}

for bus_number in bus_numbers:
    average_data = make_data.bus_average_data(get_in_off_merged_data_copy, bus_number)
    average_data_dict[bus_number] = average_data

for bus_number in bus_numbers:
    df_name = f"AVERAGE_DATA_{bus_number}"
    globals()[df_name] = average_data_dict[bus_number]

In [4]:
# 필요한 노선만 필터링한 뒤, int형으로 변환
bus_numbers = ['102', '103', '105', '106', '108', '115', '119', '201', '211', '301', '311', '314', '511', '603', '604', '605', '613', '615', '703', '704', '705', '706', '802']
ROUTE_BUS_NUMBER = getrouteinfoall_df[getrouteinfoall_df['ROUTE_NO'].isin(bus_numbers)]
ROUTE_BUS_NUMBER['ROUTE_NO'] = ROUTE_BUS_NUMBER['ROUTE_NO'].astype(int)

In [5]:
bus_numbers = [102, 103, 105, 106, 108, 115, 119, 201, 211, 301, 311, 314, 511, 603, 604, 605, 613, 615, 703, 704, 705, 706, 802] # 위에서 str타입으로 불러왔으니 다시 불러오기

for bus_number in bus_numbers: # 노선(버스)별로 BUSSTOP_TP를 필터링해줌
    variable_name = f"BUS_{bus_number}_BUSSTOP_TP" # BUS_102_BUSSTOP_TP
    globals()[variable_name] = make_data.get_bus_stop_tp(getstationbyrouteall_df1, ROUTE_BUS_NUMBER, bus_number)

In [6]:
# 각 버스별로 상/하행 데이터 생성
for bus_number in bus_numbers:
    average_data = globals().get(f'AVERAGE_DATA_{bus_number}')  # ex: AVERAGE_DATA_102
    busstop_tp_data = globals().get(f'BUS_{bus_number}_BUSSTOP_TP')  # ex: BUS_102_BUSSTOP_TP
    
    bus_number_str = str(bus_number).zfill(3)  # 버스번호를 3자리로 맞춤
    bus_name = f'BUS_{bus_number_str}'
    
    # 상/하행 데이터 생성
    up_data, down_data = make_data.final_data(average_data, busstop_tp_data)
    # 데이터 저장
    globals()[f'{bus_name}_UP'] = up_data
    globals()[f'{bus_name}_DOWN'] = down_data

In [10]:
# def plot_bus_heatmap(bus_number, direction):
#     plt.rc('font', family='malgun gothic')
#     plt.figure(figsize=(20, 20))
    
#     if direction == 'UP':
#         heatmap_data = globals().get(f'BUS_{bus_number}_UP')
#     elif direction == 'DOWN':
#         heatmap_data = globals().get(f'BUS_{bus_number}_DOWN')
    
#     heatmap_data = heatmap_data.iloc[:, :19]  # 좌표 데이터 제거 후 히트맵 표시
    
#     sns.heatmap(heatmap_data, annot=True, fmt='.1f', vmin=0, vmax=60)
#     plt.title(f'{bus_number}번 {direction} 버스 히트맵')
#     plt.xlabel('시간대')
#     plt.ylabel('정류장순번')

#      # 그래프를 이미지 파일로 저장
#     image_path = f'./IMAGE_FILE_3/{bus_number}_{direction}_heatmap.png'
#     plt.savefig(image_path, bbox_inches='tight')

#     plt.show()

In [9]:
# # 버스 전체 히트맵 저장
# bus_numbers = [102, 103, 105, 106, 108, 115, 119, 201, 211, 301, 311, 314, 511, 603, 604, 605, 613, 615, 703, 704, 705, 706, 802]
# up_down = ['UP', 'DOWN']

# def plot_and_save_heatmaps():
#     for bus_number in bus_numbers:
#         for direction in up_down:
#             plot_bus_heatmap(bus_number, direction)
#             print(f'{bus_number}번 {direction} 버스 히트맵 생성 및 저장 완료.')

# # 이미지 파일로 저장
# plot_and_save_heatmaps()

## folium heatmapwithtime

speed

In [7]:
import pandas as pd

merged_df = pd.read_csv('./merged_df.csv', encoding = 'cp949')
merged_df = data_preprocessing.merged_df_preprocessing(merged_df)
merged_df.head(2)

,노선,정류장구간,5:00,6:00,7:00,8:00,9:00,10:00,11:00,12:00,13:00,14:00,15:00,16:00,17:00,18:00,19:00,20:00,21:00,22:00,23:00,평균값
0,102,수통골삼거리-41670▶한밭대학교-41690,24.0,28.0,17.5,16.0,18.8,19.5,19.3,18.8,18.8,21.3,21.8,19.5,18.0,17.3,15.0,21.0,23.0,20.3,17.0,19.7
1,102,한밭대학교-41690▶삼성화재연수원-41730,25.3,30.8,31.0,27.5,27.8,26.5,21.3,21.5,23.8,21.3,21.5,19.8,20.3,19.8,25.5,22.5,23.0,20.5,21.0,23.7


In [8]:
# 버스별 데이터프레임 생성 코드

times = list(range(6, 23))
bus_numbers = [102, 103, 105, 106, 108, 115, 119, 201, 211, 301, 311, 314, 511, 603, 604, 605, 613, 615, 703, 704, 705, 706, 802] # 위에서 str타입으로 불러왔으니 다시 불러오기

for bus_number in bus_numbers:
    variable_name = f"BUS_SPEED_{bus_number}_MEAN" # BUS_SPEED_102_MEAN
    globals()[variable_name] = make_data.bus_speed(merged_df, bus_number)
    
for bus_number in bus_numbers:
    variable_name = f"BUS_SPEED_ROUTE_{bus_number}" # BUS_SPEED_ROUTE_102
    globals()[variable_name] = make_data.get_bus_stop_tp(getstationbyrouteall_df2, ROUTE_BUS_NUMBER, bus_number)
    
for bus_number in bus_numbers:
    variable_name = f'BUS_SPEED_{bus_number}_MEAN_COORDINATE' # BUS_SPEED_102_MEAN_COORDINATE
    mean_variable = globals()[f'BUS_SPEED_{bus_number}_MEAN'] 
    route_variable = globals()[f'BUS_SPEED_ROUTE_{bus_number}']
    globals()[variable_name] = make_data.make_coordinate(mean_variable, route_variable)
    
for bus_number in bus_numbers:
    for time in times:
        variable_name = f'BUS_SPEED_{bus_number}_MEAN_COORDINATE'
        coordinate_data = globals()[variable_name]
        final_data_variable_name = f'BUS_SPEED_{bus_number}_MEAN_COORDINATE_{time}' # BUS_SPEED_102_MEAN_COORDINATE_7
        globals()[final_data_variable_name] = make_data.speed_final_data(coordinate_data, time)
        
for bus_number in bus_numbers:
    for time in times:
        variable_name = f'BUS_SPEED_{bus_number}_MEAN_COORDINATE_{time}'
        up_variable_name = f'BUS_SPEED_{bus_number}_MEAN_COORDINATE_{time}_UP'
        down_variable_name = f'BUS_SPEED_{bus_number}_MEAN_COORDINATE_{time}_DOWN' # BUS_SPEED_102_MEAN_COORDINATE_UP
        
        coordinate_data = globals()[variable_name]
        up_data, down_data = make_data.make_up_down(coordinate_data)
        
        globals()[up_variable_name] = up_data
        globals()[down_variable_name] = down_data

## 시각화

In [19]:
# import folium
# from shapely.wkt import loads
# import pandas as pd

# # 지도의 중심 좌표 설정
# map_center = [BUS_704_UP_7['GPS_LATI'].mean(), BUS_704_UP_7['GPS_LONG'].mean()]

# # 지도 생성
# m = folium.Map(location=map_center, zoom_start=12)

# # 인원 수 범위에 따른 색상 지정 함수
# def get_color(passengers):
#     if passengers <= 15:
#         return 'green'  # 여유
#     elif passengers <= 30:
#         return 'yellow'  # 보통
#     else:
#         return 'red'  # 혼잡

# colormap = {
#     'slow': 'red',
#     'medium': 'green',
#     'fast': 'blue'
# }

# # Linestring 지도에 표시하는 코드
# for idx, row in BUS_SPEED_704_MEAN_COORDINATE_7_UP.iterrows():
#     if row['LineString'] is not None:
#         line = loads(row['LineString'])
#         coords = line.coords
#         color = colormap.get(row['speed_range'], 'gray') # speed_range에 따른 색상 설정, 미지정 시 회색 사용
#         folium.PolyLine(locations=coords, color=color, weight=7, popup=f"Speed: {row['17시']} km/h").add_to(m) # PolyLine 객체를 생성하여 지도에 추가하고 팝업으로 속도 정보를 표시

# # 정류장별 원 추가
# for index, row in BUS_704_UP_7.iterrows():
#     latitude = row['GPS_LATI']
#     longitude = row['GPS_LONG']
#     passengers = row['7']
    
#     # 원의 색상 설정
#     color = get_color(passengers)
    
#     # 원을 지도에 추가
#     folium.CircleMarker(
#         location=[latitude, longitude],
#         radius=2,  # 원의 크기
#         color=color,
#         fill=True,
#         fill_color=color,
#         popup=f"정류장: {index}<br>인원: {passengers}명"
#     ).add_to(m)

# # 범례 (legend) 추가해주는 코드
# legend_html = """
# <div style="position: fixed; bottom: 50px; left: 50px; width: 100px; height: 100px; 
#      border:2px solid grey; z-index:9999; font-size:14px;
#      background-color:white; opacity: 0.9;">
#      <p style="margin: 5px;"><span style="color: red;">&#9679;</span> Slow</p>
#      <p style="margin: 5px;"><span style="color: green;">&#9679;</span> Medium</p>
#      <p style="margin: 5px;"><span style="color: blue;">&#9679;</span> Fast</p>
# </div>
# """
# m.get_root().html.add_child(folium.Element(legend_html))

# # 지도를 HTML 파일로 저장하거나 직접 출력할 수 있습니다
# # m.save('combined_map.html')
# m


In [23]:
# 출근시간대 7-9, 퇴근시간대 16-18
BUS_704_UP_7 = BUS_704_UP[['7','GPS_LATI','GPS_LONG']]
BUS_704_UP_8 = BUS_704_UP[['8','GPS_LATI','GPS_LONG']]
BUS_704_UP_9 = BUS_704_UP[['9','GPS_LATI','GPS_LONG']]

BUS_704_DOWN_7 = BUS_704_DOWN[['7','GPS_LATI','GPS_LONG']]
BUS_704_DOWN_8 = BUS_704_DOWN[['8','GPS_LATI','GPS_LONG']]
BUS_704_DOWN_9 = BUS_704_DOWN[['9','GPS_LATI','GPS_LONG']]

BUS_705_UP_7 = BUS_705_UP[['7','GPS_LATI','GPS_LONG']]
BUS_705_UP_8 = BUS_705_UP[['8','GPS_LATI','GPS_LONG']]
BUS_705_UP_9 = BUS_705_UP[['9','GPS_LATI','GPS_LONG']]

BUS_705_DOWN_7 = BUS_705_DOWN[['7','GPS_LATI','GPS_LONG']]
BUS_705_DOWN_8 = BUS_705_DOWN[['8','GPS_LATI','GPS_LONG']]
BUS_705_DOWN_9 = BUS_705_DOWN[['9','GPS_LATI','GPS_LONG']]

BUS_802_UP_7 = BUS_802_UP[['7','GPS_LATI','GPS_LONG']]
BUS_802_UP_8 = BUS_802_UP[['8','GPS_LATI','GPS_LONG']]
BUS_802_UP_9 = BUS_802_UP[['9','GPS_LATI','GPS_LONG']]

BUS_802_DOWN_7 = BUS_802_DOWN[['7','GPS_LATI','GPS_LONG']]
BUS_802_DOWN_8 = BUS_802_DOWN[['8','GPS_LATI','GPS_LONG']]
BUS_802_DOWN_9 = BUS_802_DOWN[['9','GPS_LATI','GPS_LONG']]

In [49]:
# 출근시간대 7-9, 퇴근시간대 16-18
BUS_704_UP_16 = BUS_704_UP[['16','GPS_LATI','GPS_LONG']]
BUS_704_UP_17 = BUS_704_UP[['17','GPS_LATI','GPS_LONG']]
BUS_704_UP_18 = BUS_704_UP[['18','GPS_LATI','GPS_LONG']]

BUS_704_DOWN_16 = BUS_704_DOWN[['16','GPS_LATI','GPS_LONG']]
BUS_704_DOWN_17 = BUS_704_DOWN[['17','GPS_LATI','GPS_LONG']]
BUS_704_DOWN_18 = BUS_704_DOWN[['18','GPS_LATI','GPS_LONG']]

BUS_705_UP_16 = BUS_705_UP[['16','GPS_LATI','GPS_LONG']]
BUS_705_UP_17 = BUS_705_UP[['17','GPS_LATI','GPS_LONG']]
BUS_705_UP_18 = BUS_705_UP[['18','GPS_LATI','GPS_LONG']]

BUS_705_DOWN_16 = BUS_705_DOWN[['16','GPS_LATI','GPS_LONG']]
BUS_705_DOWN_17 = BUS_705_DOWN[['17','GPS_LATI','GPS_LONG']]
BUS_705_DOWN_18 = BUS_705_DOWN[['18','GPS_LATI','GPS_LONG']]

BUS_802_UP_16 = BUS_802_UP[['16','GPS_LATI','GPS_LONG']]
BUS_802_UP_17 = BUS_802_UP[['17','GPS_LATI','GPS_LONG']]
BUS_802_UP_18 = BUS_802_UP[['18','GPS_LATI','GPS_LONG']]

BUS_802_DOWN_16 = BUS_802_DOWN[['16','GPS_LATI','GPS_LONG']]
BUS_802_DOWN_17 = BUS_802_DOWN[['17','GPS_LATI','GPS_LONG']]
BUS_802_DOWN_18 = BUS_802_DOWN[['18','GPS_LATI','GPS_LONG']]

In [78]:
import folium
from shapely.wkt import loads

# 지도의 중심 좌표 설정
map_center = [BUS_802_UP_18['GPS_LATI'].mean(), BUS_802_UP_18['GPS_LONG'].mean()]

# 지도 생성
m = folium.Map(location=map_center, tiles='cartodbdark_matter', zoom_start=12)

colormap = {
    'slow': 'red',
    'medium': 'orange',
    'fast': 'blue'
}

# 인원 수 범위에 따른 색상 지정 함수
def get_color(passengers):
    if passengers <= 15:
        return 'green'
    elif passengers <= 30:
        return 'yellow'
    else:
        return 'red'

# Linestring 지도에 표시하는 코드
for idx, row in BUS_SPEED_802_MEAN_COORDINATE_18_UP.iterrows():
    if row['LineString']:
        line = loads(row['LineString'])
        coords = line.coords
        color = colormap.get(row['speed_range'], 'gray')
        popup_content = f"Speed: {row['18시']} km/h"
        folium.PolyLine(locations=coords, color=color, weight=7, popup=popup_content).add_to(m)

# 정류장별 원 추가
for index, row in BUS_802_UP_18.iterrows():
    latitude, longitude, passengers = row['GPS_LATI'], row['GPS_LONG'], row['18']
    color = get_color(passengers)
    folium.CircleMarker(location=[latitude, longitude], radius=5, color=color, fill=True, fill_color=color, popup=f"정류장: {index}<br>인원: {passengers}명").add_to(m)

# 범례 (legend) 추가해주는 코드
legend_html = """
<div style="position: fixed; bottom: 50px; left: 50px; width: 100px; height: 100px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white; opacity: 0.9;">
     <p style="margin: 5px;"><span style="color: red;">&#9679;</span> Slow</p>
     <p style="margin: 5px;"><span style="color: orange;">&#9679;</span> Medium</p>
     <p style="margin: 5px;"><span style="color: blue;">&#9679;</span> Fast</p>
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

# 지도를 HTML 파일로 저장하거나 직접 출력할 수 있습니다
# m.save('combined_map.html')
m.save('./HTML/퇴근/BUS_802_UP_18.html')


In [86]:
import folium
from shapely.wkt import loads

# 지도의 중심 좌표 설정
map_center = [BUS_704_DOWN_18['GPS_LATI'].mean(), BUS_704_DOWN_18['GPS_LONG'].mean()]

# 지도 생성
m = folium.Map(location=map_center, tiles='cartodbdark_matter', zoom_start=12)

colormap = {
    'slow': 'red',
    'medium': 'orange',
    'fast': 'blue'
}

# 인원 수 범위에 따른 색상 지정 함수
def get_color(passengers):
    if passengers <= 15:
        return 'green'
    elif passengers <= 30:
        return 'yellow'
    else:
        return 'red'

# Linestring 지도에 표시하는 코드
for idx, row in BUS_SPEED_704_MEAN_COORDINATE_18_DOWN.iterrows():
    if row['LineString']:
        line = loads(row['LineString'])
        coords = line.coords
        color = colormap.get(row['speed_range'], 'gray')
        popup_content = f"Speed: {row['18시']} km/h"
        folium.PolyLine(locations=coords, color=color, weight=7, popup=popup_content).add_to(m)

# 정류장별 원 추가
for index, row in BUS_704_DOWN_18.iterrows():
    latitude, longitude, passengers = row['GPS_LATI'], row['GPS_LONG'], row['18']
    color = get_color(passengers)
    folium.CircleMarker(location=[latitude, longitude], radius=5, color=color, fill=True, fill_color=color, popup=f"정류장: {index}<br>인원: {passengers}명").add_to(m)

# 범례 (legend) 추가해주는 코드
legend_html = """
<div style="position: fixed; bottom: 50px; left: 50px; width: 100px; height: 100px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white; opacity: 0.9;">
     <p style="margin: 5px;"><span style="color: red;">&#9679;</span> Slow</p>
     <p style="margin: 5px;"><span style="color: orange;">&#9679;</span> Medium</p>
     <p style="margin: 5px;"><span style="color: blue;">&#9679;</span> Fast</p>
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

# 지도를 HTML 파일로 저장하거나 직접 출력할 수 있습니다
# m.save('combined_map.html')
m.save('./HTML/퇴근/BUS_704_DOWN_18.html')


### 출퇴근 시간대 노선
> 105번, 604번, 605번, 703번, 705번

# [704,705,802]

In [8]:
import pandas as pd
import folium
from folium import plugins
from sklearn.preprocessing import MinMaxScaler

minMaxScaler = MinMaxScaler()

BUS_NUM_6_TO_22 = [704, 705, 802]
UP_DOWN_6_TO_22 = ['UP', 'DOWN']
time_series = ['6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22']

for bus_num in BUS_NUM_6_TO_22:
    for up_down in UP_DOWN_6_TO_22:
        # min-max 스케일링
        bus_num_str = str(bus_num).zfill(3) # 3자리 숫자로 맞춰줌
        data_to_scale = eval(f'BUS_{bus_num_str}_{up_down}.iloc[:,:19]')  # Extract the data for scaling
        scaled_data = minMaxScaler.fit_transform(data_to_scale)  # Fit and transform the scaler
        BUS_minMaxScaled = pd.DataFrame(scaled_data, index=data_to_scale.index, columns=data_to_scale.columns)
        BUS_concat = pd.concat([BUS_minMaxScaled, eval(f'BUS_{bus_num_str}_{up_down}.iloc[:, 19:]')], axis=1)
        
        # 히트맵 데이터 생성
        heat_data = [[[row['GPS_LATI'], row['GPS_LONG'], row[time]] for index, row in BUS_concat.iterrows()] for time in time_series]
        
        # 히트맵 folium 지도 생성 및 저장
        m = folium.Map(location=[36.3504119, 127.3845475], zoom_start=12)
        heat_map = plugins.HeatMapWithTime(heat_data, index=[time for time in time_series], auto_play=False, radius=20, max_opacity=1, gradient={0.1: 'blue', 0.3: 'lime', 0.6: 'yellow', 1: 'red'})
        heat_map.add_to(m)
        m.save(f'./HEATMAP_HTML_FILE_4/BUS_{bus_num_str}_{up_down}.html')
        # await map_to_png(f'./HEATMAP_CAPTURE/BUS_{bus_num_str}_{up_down}', m) # 출처 : https://yeomss.tistory.com/276

        # 색상 다르게 하려면 gradient = {0.2: '#FBD973',  0.4: '#fa782f', 0.75: '#F16578', 1: '#782890'})